In [3]:
from sklearn.datasets import load_diabetes
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [9]:
X, y= load_diabetes(return_X_y=True)

In [10]:
def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    param ={
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.09),
        "depth": trial.suggest_int("depth", 1, 10),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
    }

    cb = CatBoostClassifier(**param)
    cb.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=100)
    preds = cb.predict(X_test)
    return preds

In [11]:
study = optuna.create_study(direction="maximize")

[I 2024-04-22 11:04:45,460] A new study created in memory with name: no-name-fd7a8df8-e68d-43d1-8455-9ba0d6e514ba


In [12]:
%%time

study.optimize(func=objective, n_trials=100, n_jobs=-1)

[W 2024-04-22 11:04:45,952] Trial 0 failed with parameters: {'iterations': 939, 'learning_rate': 0.07078513017545043, 'depth': 1, 'l2_leaf_reg': 9} because of the following error: CatBoostError('C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/data.cpp:198: Dataset test #0 contains class label "186" that is not present in the learn dataset').
Traceback (most recent call last):
  File "C:\Users\Panache Branding\miniconda3\envs\catboost_env\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Panache Branding\AppData\Local\Temp\ipykernel_2500\2477886805.py", line 12, in objective
    cb.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=100)
  File "C:\Users\Panache Branding\miniconda3\envs\catboost_env\lib\site-packages\catboost\core.py", line 5220, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baselin

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/data.cpp:198: Dataset test #0 contains class label "186" that is not present in the learn dataset